In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def calculate_technical_indicators(df, close_col='close', high_col='high', low_col='low', volume_col='volume'):
    """
    Calculate technical indicators for stock price analysis.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for close, high, low, and volume
    close_col (str): Name of closing price column
    high_col (str): Name of high price column
    low_col (str): Name of low price column
    volume_col (str): Name of volume column
    
    Returns:
    pandas.DataFrame: Original data with additional technical indicators
    """
    df = df.copy()
    
    # Trend Indicators
    # Moving Averages
    df['sma_5'] = df[close_col].rolling(window=5).mean()
    df['sma_20'] = df[close_col].rolling(window=20).mean()
    df['sma_50'] = df[close_col].rolling(window=50).mean()
    
    # Exponential Moving Average
    df['ema_12'] = df[close_col].ewm(span=12, adjust=False).mean()
    df['ema_26'] = df[close_col].ewm(span=26, adjust=False).mean()
    
    # MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Momentum Indicators
    # Relative Strength Index (RSI)
    delta = df[close_col].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Stochastic Oscillator
    lookback = 14
    df['lowest_low'] = df[low_col].rolling(window=lookback).min()
    df['highest_high'] = df[high_col].rolling(window=lookback).max()
    df['stoch_k'] = 100 * (df[close_col] - df['lowest_low']) / (df['highest_high'] - df['lowest_low'])
    df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()
    
    # Volatility Indicators
    # Bollinger Bands
    df['bb_middle'] = df[close_col].rolling(window=20).mean()
    df['bb_upper'] = df['bb_middle'] + 2 * df[close_col].rolling(window=20).std()
    df['bb_lower'] = df['bb_middle'] - 2 * df[close_col].rolling(window=20).std()
    
    # Average True Range (ATR)
    high_low = df[high_col] - df[low_col]
    high_close = np.abs(df[high_col] - df[close_col].shift())
    low_close = np.abs(df[low_col] - df[close_col].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['atr'] = true_range.rolling(14).mean()
    
    # Volume-based Indicators
    # On-Balance Volume (OBV)
    df['daily_ret'] = df[close_col].pct_change()
    df['obv'] = np.where(df['daily_ret'] > 0, df[volume_col], 
                        np.where(df['daily_ret'] < 0, -df[volume_col], 0)).cumsum()
    
    # Volume-Weighted Average Price (VWAP)
    df['vwap'] = (df[close_col] * df[volume_col]).cumsum() / df[volume_col].cumsum()
    
    # Price Rate of Change
    df['roc_5'] = df[close_col].pct_change(periods=5) * 100
    df['roc_20'] = df[close_col].pct_change(periods=20) * 100
    
    # Additional Derived Features
    df['price_volatility'] = df[close_col].rolling(window=20).std()
    df['volume_volatility'] = df[volume_col].rolling(window=20).std()
    
    return df